In [1]:
import pandas as pd
from groq import Groq
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import urllib.parse
import numpy as np
import re

In [2]:
df = pd.read_parquet("hf://datasets/Shiyu-Lab/Wikipedia_Person_Unlearn/forget_20_1/train-00000-of-00001.parquet")

In [3]:
client = Groq(
    api_key = "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"
)

In [ ]:
system_prompt = """
You are an expert teacher, who can create questions and answers from a given context. 
Given the user wikipedia page context about {domain_person_name}, please provide as many questions and answers possible from it.
For each section, provide atleast 2 questions and answers.
The question and answers should follow the Interrogative syntactic structure, 

The questions should be on their birth, family background, education, career, achievements and other relevant topics.

The output should be in JSON format with the following keys:

{{
    "name": name of the person,
    "question1": question1,
    "answer1": answer1,
    "section" : part of the wikipedia section,
    "difficulty" : difficulty of the question,
    "question2": question2,
    .................
}}

Please be precise with the question and answer. Do not generate any other text.
"""

In [ ]:
prompt = """
{content}
"""

In [ ]:
final_df['new_questions'] = ''
for i, row in final_df.iterrows():
    name = row['SimilarName']
    content = row['content']
    questions = row['questions']
    title = row['title']
    
    sys_prompt = system_prompt.format(domain_person_name=name)
    prompt_text = prompt.format(content=content)
    messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": prompt_text},]
    print("Now generating for ", name)
    response = client.chat.completions.create(
        messages= messages,
        model="llama-3.3-70b-versatile",
        temperature=0.1,
        max_completion_tokens=1000
    )
    response_text = response.choices[0].message.content
    final_df.at[i, 'new_questions'] = response_text
    